In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable
from classification_models import GetLoader

In [2]:
#导入数据
factor_df=pd.read_csv("/data01/data_for_intern/factor_data_13_19.csv")
return_df=pd.read_csv("/data01/data_for_intern/return_data_13_19.csv")
return_label=return_df.drop(columns=['return_adj_d001','return_adj_d005','car_hs300_b30_d005'])
return_label['car_hs300_b30_d001']=(return_label['car_hs300_b30_d001']>=0)*1
#利用内连接方式合并数据
merged_factor_return = factor_df.merge(return_label,on=["stock_code","trade_date"],how="inner")
merged_factor_return

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
0,000001.SZ,2013-01-04,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,...,0.790698,0.724551,0.812212,20.869800,43.5275,0.741100,0.680235,1.57469,3.43924,1
1,000001.SZ,2013-01-07,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,...,0.856688,0.909091,0.841909,18.507200,41.6409,0.741100,0.704779,1.47261,3.15030,0
2,000001.SZ,2013-01-08,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,...,0.676724,0.727273,0.786972,20.082300,29.5337,0.605634,0.705782,1.48008,3.02077,0
3,000001.SZ,2013-01-09,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,...,0.632231,0.642424,0.759596,8.662940,27.9438,0.536232,0.699468,1.47922,3.01475,0
4,000001.SZ,2013-01-10,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,...,0.629167,0.516667,0.628788,12.206900,27.2890,0.520000,0.690864,1.51058,3.06918,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088415,603999.SH,2019-12-13,0.063063,0.007508,0.070571,-9.0,-0.875646,-0.32,0.138746,-0.876772,...,0.542857,0.855769,0.822900,1.148900,20.9302,0.029940,0.307711,1.14262,3.09775,1
5088416,603999.SH,2019-12-16,0.038682,0.007163,0.045845,-9.0,-0.880784,-0.18,0.306574,-0.751634,...,0.583333,0.884298,0.811730,0.782227,27.0718,0.187879,0.276938,1.04110,2.77682,0
5088417,603999.SH,2019-12-17,0.002821,0.016925,0.019746,-8.0,-0.854110,0.11,0.863933,-0.479172,...,0.654971,0.778761,0.839609,0.342224,29.0801,0.298013,0.260858,1.04584,2.66659,0
5088418,603999.SH,2019-12-18,0.010029,0.012894,0.022923,-7.0,-0.733061,0.04,0.511944,-0.716503,...,0.670659,0.736364,0.799807,0.391114,29.2537,0.324324,0.252109,1.07986,2.67018,1


In [3]:
#筛选日期节点,以15个交易日为界
date=[]

In [4]:
import datetime
date.append("2019-01-21")

In [5]:
i=0
while(i>-1):
    now=datetime.datetime.strptime(date[i], "%Y-%m-%d")
    if date[i]=="2019-12-02":
        break
    delta=datetime.timedelta(days=21)
    n_days=now+delta
    date.append(n_days.strftime('%Y-%m-%d'))
    i=i+1

In [6]:
date

['2019-01-21',
 '2019-02-11',
 '2019-03-04',
 '2019-03-25',
 '2019-04-15',
 '2019-05-06',
 '2019-05-27',
 '2019-06-17',
 '2019-07-08',
 '2019-07-29',
 '2019-08-19',
 '2019-09-09',
 '2019-09-30',
 '2019-10-21',
 '2019-11-11',
 '2019-12-02']

In [7]:
idx_name=np.unique(merged_factor_return["stock_code"])
idx_name

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '603997.SH',
       '603998.SH', '603999.SH'], dtype=object)

In [ ]:
def data_cleaning(merged_factor_return,date1,date2,date3,idx_name, gate):
    #data cleaning
    if gate == 0 :
        train_data=np.zeros([1,10,42])
        valid_data=np.zeros([1,10,42])
    
        df_data=merged_factor_return.drop(['car_hs300_b30_d001'],axis=1)
    
        #training data
        #waste=[]
        for name in idx_name:
            temp=df_data[(df_data["stock_code"]==name)]
            d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date1)].index
            if d1.size==0:
                #waste.append(name)
                continue
            d1=int(np.array(d1))
            d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date2)].index
            if d2.size==0:
                #waste.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                train_temp=temp.loc[i:i+9].values.reshape(1,10,42)
                train_data=np.concatenate((train_data,train_temp),axis=0)
            
        #waste2=[]
        for name in idx_name:
            temp=df_data[(df_data["stock_code"]==name)]
            d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date2)].index
            if d1.size==0:
                #waste2.append(name)
                continue
            d1=int(np.array(d1))
            d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date3)].index
            if d2.size==0:
                #waste2.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                valid_temp=temp.loc[i:i+9].values.reshape(1,10,42)
                valid_data=np.concatenate((valid_data,valid_temp),axis=0)
    
        #label cleaning
        label=merged_factor_return[["car_hs300_b30_d001","stock_code","trade_date"]]

        train_label=np.zeros([1,])
        valid_label=np.zeros([1,])
    
        #waste=[]
        for name in idx_name:
            temp=label[(label["stock_code"]==name)]
            d1=label[(label["stock_code"]==name) & (label["trade_date"]==date1)].index
            if d1.size==0:
                #waste.append(name)
                continue
            d1=int(np.array(d1))
            d2=label[(label["stock_code"]==name) & (label["trade_date"]==date2)].index
            if d2.size==0:
                #waste.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                train_temp=temp.loc[i+10].values
                train_label=np.concatenate((train_label,train_temp),axis=0)
    
        #waste=[]
        for name in idx_name:
            temp=label[(label["stock_code"]==name)]
            d1=label[(label["stock_code"]==name) & (label["trade_date"]==date2)].index
            if d1.size==0:
                #waste.append(name)
                continue
            d1=int(np.array(d1))
            d2=label[(label["stock_code"]==name) & (label["trade_date"]==date3)].index
            if d2.size==0:
                #waste.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                valid_temp=temp.loc[i+10].values
                valid_label=np.concatenate((valid_label,valid_temp),axis=0)
            
        train_data=np.delete(train_data,0,axis=0)
        valid_data=np.delete(valid_data,0,axis=0)
        train_label=np.delete(train_label,0,axis=0)
        valid_label=np.delete(valid_label,0,axis=0)
        
        np.save("data"+str(gate),train_data)
        np.save("label"+str(gate),train_label)
        np.save("data"+str(gate+1),valid_data)        
        np.save("label"+str(gate+1),valid_label)    
        

        
    else:
        
        train_data = np.load("data"+str(gate)+".npy")
        train_label = np.load("label"+str(gate)+".npy")    

        valid_data=np.zeros([1,10,42])
        df_data=merged_factor_return.drop(['car_hs300_b30_d001'],axis=1)
        for name in idx_name:
            temp=df_data[(df_data["stock_code"]==name)]
            d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date2)].index
            if d1.size==0:
                #waste2.append(name)
                continue
            d1=int(np.array(d1))
            d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date3)].index
            if d2.size==0:
                #waste2.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                valid_temp=temp.loc[i:i+9].values.reshape(1,10,42)
                valid_data=np.concatenate((valid_data,valid_temp),axis=0)

        label=merged_factor_return[["car_hs300_b30_d001","stock_code","trade_date"]]
        valid_label=np.zeros([1,])

        for name in idx_name:
            temp=label[(label["stock_code"]==name)]
            d1=label[(label["stock_code"]==name) & (label["trade_date"]==date2)].index
            if d1.size==0:
                #waste.append(name)
                continue
            d1=int(np.array(d1))
            d2=label[(label["stock_code"]==name) & (label["trade_date"]==date3)].index
            if d2.size==0:
                #waste.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2-9):
                valid_temp=temp.loc[i+10].values
                valid_label=np.concatenate((valid_label,valid_temp),axis=0)

        valid_data=np.delete(valid_data,0,axis=0)
        valid_label=np.delete(valid_label,0,axis=0)            
        np.save("data"+str(gate+1),valid_data)        
        np.save("label"+str(gate+1),valid_label)
        
    return train_data, valid_data, train_label, valid_label

In [ ]:
#完成数据清洗的函数后开始定义模型

In [ ]:
#简单LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes,device=torch.device("cuda:1")):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  # batch_first=True仅仅针对输入而言
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 设置初始状态h_0与c_0的状态是初始的状态，一般设置为0，尺寸是,x.size(0)
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Forward propagate RNN
        out, (h_n, c_n) = self.lstm(x, (h0, c0))  # 送入一个初始的x值，作为输入以及(h0, c0)

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  # output也是batch_first, 实际上h_n与c_n并不是batch_first
        return out
    
#搭建第一类ResNet block
class BasicBlock(nn.Module):#基本残差网络的一个模块类
    expansion = 1#每一个residual block中不改变width,height,channel数，即增加的residual部分不需要做卷积处理

    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)#stride=1,kernel_size=3,padding=1保证了data的
        self.drop = nn.Dropout(0.7)                                                                           #size不变 
        self.bn1 = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channel)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:  #BasicBlock内不需要调整residual的height,width,channel
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out


#dropout resnet+lstm
#组建block成ResNet
class ResNet1D_LSTM(nn.Module):
    def __init__(self, block, blocks_num, num_classes, feature_channel,hidden_size, num_layers,device ):
        super(ResNet1D_LSTM, self).__init__()
        self.in_channel = 64
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.conv1 = nn.Conv1d(in_channels=feature_channel, out_channels=self.in_channel, kernel_size=2, stride=2,
                               padding=0, bias=False)
        self.bn1 = nn.BatchNorm1d(self.in_channel)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)  #channel数变为n/2(非整数时向下取整)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])         #按照已有结论按二次方形式增长ResNet不同阶段的channel
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)#stride=2表示想把上一个layer传过来的size缩减为1/2
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        #self.avgpool = nn.AdaptiveAvgPool1d((1, 1))  # output size = (1, 1) 
        self.lstm = nn.LSTM(256, hidden_size, num_layers, batch_first=True)  # batch_first=True仅仅针对输入而言
        self.fc = nn.Linear(hidden_size, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight.to(device), mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x) #预处理
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x) #残差网络
        x = self.layer2(x)
        x = self.layer3(x)
        
        x=torch.transpose(x,2,1)
        #因为pytorch里lstm和conv1d的input sequence位置不一样，需要调整。
        
        # 设置初始状态h_0与c_0的状态是初始的状态，一般设置为0，尺寸是,x.size(0)
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Forward propagate RNN
        out, (h_n, c_n) = self.lstm(x, (h0, c0))  # 送入一个初始的x值，作为输入以及(h0, c0)

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  # output也是batch_first, 实际上h_n与c_n并不是batch_first
        return out

In [ ]:
#预测
def predict_precision(model,images,labels,device,predict_type):
    model.eval()
    with torch.no_grad():
        correct=0
        total=0
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=sum(predicted)
        correct+=(sum(predicted*labels))
        print('precision of the model on the'+predict_type+'data: {}%'.format(100*correct/total))
    model.train()
    return predicted, 100*correct/total

In [ ]:
#训练
def training(model, num_epochs, train_loader,valid_loader, loss_function, optimizer, device):
    total_step=0
    train_precision=[]
    valid_precision=[]
    for epoch in range(num_epochs):
        for data in train_loader:
            images, labels = data
            images=images.type(torch.FloatTensor)
            labels=labels.type(torch.FloatTensor)
            images=images.to(device)
            labels=labels.to(device)
            #forward pass
            outputs=model(images)
            loss=loss_function(outputs,labels.long())
        
            #Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_step+=1
            if (total_step)%100==0:
                #print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))
                _,train_pre=predict_precision(model,images,labels,device,predict_type='training')
                train_precision.append(train_pre)
                for images, labels in valid_loader:
                    _,valid_pre=predict_precision(model,images,labels,device,predict_type='validation')
                valid_precision.append(valid_pre)
    
    return train_precision, valid_precision, model

In [ ]:
def nan_dele(data, label):
    a=data
    b=a.reshape([a.shape[0],a.shape[1]*a.shape[2]])
    c=b[~np.isnan(b).any(axis=1),:]
    data=c.reshape([c.shape[0], a.shape[1],a.shape[2]])
    d=label.reshape(-1,1)
    label=d[~np.isnan(b).any(axis=1),:].reshape(-1,)
    return data, label

In [ ]:
def model_test(model, merged_factor_return, date, num_epochs, loss_function, optimizer, batch_size, device, data_reverse):
    idx_name=np.unique(merged_factor_return["stock_code"])
    sum_train_precision=[]
    sum_valid_precision=[]
    for i in range( len(date)-2):
        print(i)
        date1=date[i]
        date2=date[i+1]
        date3=date[i+2]
        train_data, valid_data, train_label, valid_label = data_cleaning(merged_factor_return,date1,date2,date3,idx_name,i)
        train_data, train_label = nan_dele(train_data, train_label)
        valid_data, valid_label = nan_dele(valid_data, valid_label)
        if train_data.size==0:
            print("waste",date[i])
            continue
        if data_reverse == 1:
            train_data=np.transpose(train_data,(0,2,1))
            valid_data=np.transpose(valid_data,(0,2,1))
        train=GetLoader(train_data,train_label)
        valid=GetLoader(valid_data,valid_label)
        train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=0)
        valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=valid_data.shape[0],shuffle=False,num_workers=0)
        train_precision, valid_precision, model = training(model, num_epochs, train_loader,valid_loader, loss_function, optimizer, device)
        sum_train_precision.append(train_precision)
        sum_valid_precision.append(valid_precision)
        print(sum_train_precision)
        np.save('sum_train_precision2',sum_train_precision)
        np.save('sum_valid_precision2',sum_valid_precision)
        
    return sum_train_precision, sum_valid_precision

In [ ]:
sequence_length = 10  # 序列长度，将图像的每一列作为一个序列
input_size = 42  # 输入数据的维度
hidden_size = 64  # 隐藏层的size
num_layers =  4 # 有多少层

num_classes = 2
batch_size = 256
num_epochs = 300
learning_rate = 0.001

device = torch.device("cuda:1")

In [ ]:
lstm = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes,device=device)
lstm.to(device)

In [ ]:
# Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [ ]:
sum_train_precision, sum_valid_precision = model_test(lstm, merged_factor_return, date, num_epochs, loss_function, optimizer, batch_size, device, 0)

In [ ]:
#整理datacleaning函数

In [ ]:
for i in range(0,15):
    temp=np.load("label"+str(i)+".npy")
    print(i,":",temp.shape)

In [8]:
def valid_data_cleaning(merged_factor_return,date,idx_name, gate):
    #data cleaning
    date2, date3 = date
    if date3!='2019-10-07':

        print(gate)   
        valid_data=np.zeros([1,10,42])
    
        df_data=merged_factor_return.drop(['car_hs300_b30_d001'],axis=1)
    
     
        #waste2=[]
        for name in idx_name:
            #print(name)
            temp=df_data[(df_data["stock_code"]==name)]           
            d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date2)].index
            if d1.size==0:
                #waste2.append(name)
                continue
            d1=int(np.array(d1))
            #print('d1')
            d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]==date3)].index
            if d2.size==0:
                 #waste2.append(name)
                continue
            d2=int(np.array(d2))
            #print(range(d1,d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2):
                valid_temp=temp.loc[i:i+9].values.reshape(1,10,42)
                #print(valid_temp)
                valid_data=np.concatenate((valid_data,valid_temp),axis=0)
            #print(valid_data.shape)

        valid_data=np.delete(valid_data,0,axis=0)
        np.save("new_valid_data"+str(gate),valid_data)
    
        label=merged_factor_return[["car_hs300_b30_d001","stock_code","trade_date"]]
        valid_label=np.zeros([1,])

        #waste=[]
        for name in idx_name:
            temp=label[(label["stock_code"]==name)]
            d1=label[(label["stock_code"]==name) & (label["trade_date"]==date2)].index
            if d1.size==0:
                #waste.append(name)
                continue
            d1=int(np.array(d1))
            d2=label[(label["stock_code"]==name) & (label["trade_date"]==date3)].index
            if d2.size==0:
                #waste.append(name)
                continue
            d2=int(np.array(d2))
            temp=temp.drop(["stock_code","trade_date"],axis=1)
            for i in range(d1,d2):
                valid_temp=temp.loc[i+10].values
                valid_label=np.concatenate((valid_label,valid_temp),axis=0)


        valid_label=np.delete(valid_label,0,axis=0)
                
        np.save("new_valid_label"+str(gate),valid_label)    
        

In [9]:
valid_date=[]
i=2
while(i<=len(date)):
    now=datetime.datetime.strptime(date[i-1], "%Y-%m-%d")
    delta=datetime.timedelta(days=2)
    n_days=now+delta
    valid_date.append((date[i-1],n_days.strftime('%Y-%m-%d')))
    i=i+1

In [10]:
valid_date, len(valid_date), valid_date[0]

([('2019-02-11', '2019-02-13'),
  ('2019-03-04', '2019-03-06'),
  ('2019-03-25', '2019-03-27'),
  ('2019-04-15', '2019-04-17'),
  ('2019-05-06', '2019-05-08'),
  ('2019-05-27', '2019-05-29'),
  ('2019-06-17', '2019-06-19'),
  ('2019-07-08', '2019-07-10'),
  ('2019-07-29', '2019-07-31'),
  ('2019-08-19', '2019-08-21'),
  ('2019-09-09', '2019-09-11'),
  ('2019-09-30', '2019-10-02'),
  ('2019-10-21', '2019-10-23'),
  ('2019-11-11', '2019-11-13'),
  ('2019-12-02', '2019-12-04')],
 15,
 ('2019-02-11', '2019-02-13'))

In [15]:
for i in range(len(valid_date)):
    valid_data_cleaning(merged_factor_return,valid_date[i],idx_name, i)

In [27]:
np.load('new_valid_data0.npy').shape

(17920, 10, 42)

In [ ]:
og=[]
for i in range(len(valid_date)):
    _,a=valid_date[i]
    og.append(a)

In [ ]:
def waste_date(date,merged_factor_return):
    waste=[]
    for i in range(len(date)-2):
        temp=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ")].drop(['car_hs300_b30_d001'],axis=1)
        d1=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ") & (merged_factor_return["trade_date"]==date[i])].index
        if d1.size==0:
            print("waste",date[i])
            waste.append(date[i])
            continue
        d1=int(np.array(d1))
        d2=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ") & (merged_factor_return["trade_date"]==date[i])].index
        if d2.size==0:
            print("waste",date[i])
            waste.append(date[i])
            continue
        d2=int(np.array(d2))
        
    return waste

In [ ]:
waste_date(og,merged_factor_return)

In [ ]:
m=np.random.rand(9,10,4)

In [ ]:
        m=np.delete(m,0,axis=0)


In [ ]:
m.shape